Extract the Data from Amazon

In [1]:
import os
import pandas as pd
# Find the latest version of spark 3.2.x  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.2.3'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:13 http://archive

In [2]:
# Install the PostgreSQL driver in our Colab environment
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-30 04:30:14--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.94MB/s    in 0.2s    

2022-11-30 04:30:15 (4.94 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Establish a Spark session and add the Postgres driver to the filepath
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read the first CSV file from an S3 bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
toy_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Toys_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)



# Show DataFrame
toy_data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

In [5]:
# Drop null values
toy_df = toy_data_df.dropna(how='any')

#Drop duplicate values
toy_df= toy_df.drop_duplicates()

Count the number of rows

In [6]:
toy_df.count()

4863497

In [7]:
# Create the review_id_table dataframe for the database
review_id_df = toy_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R10010KWXLSPF3|   43750961|B004UUHX9M|     247428181| 2014-12-05|
|R100BG8WSTAYPG|   37257103|B00ENKGLHO|       9108602| 2014-04-29|
|R100F00UQBXLAL|   14056891|B004O3CSX6|     579644974| 2011-11-11|
|R1010FUSHO8ASL|   30419449|B000VXNYQI|     496402705| 2012-12-17|
|R101LBRJ366XL1|   36812928|B000JFJC4G|     870047482| 2015-02-28|
|R101LKRSNW6OIK|   28394801|B000FPCWXI|     841597184| 2014-07-06|
|R101TEA798DC3M|   19838846|B0017VO5AW|     484336160| 2015-06-14|
|R101UKS45S1Z0O|   18589384|B000GHWTS8|     632533309| 2009-12-12|
|R101Y71FKFPSEO|   17270128|B004U7JJWE|     366785521| 2013-03-05|
|R102ETLR4XJZFM|   37637960|B000S94NWE|     327618371| 2013-11-08|
|R102IJ4R9UE6QH|   21873089|B00HTMCF0U|     841568092| 2015-02-16|
|R1038ZA60A3YYJ|    2395887|B004S8F7QM|     829220659| 2014-11

In [8]:
review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: string (nullable = true)



In [9]:
# Create a product dataframe for the database
product_df = toy_df.select(["product_id", "product_title"])
product_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B004UUHX9M|Star Wars Deluxe ...|
|B00ENKGLHO|Ducati 1199 Panig...|
|B004O3CSX6|Gund Peek A Boo Bear|
|B000VXNYQI|Rhode Island Nove...|
|B000JFJC4G|Toysmith Rainbow ...|
|B000FPCWXI|Melissa & Doug La...|
|B0017VO5AW|Boochie, A Whole ...|
|B000GHWTS8|Reeve and Jones D...|
|B004U7JJWE|     Munchkin Deluxe|
|B000S94NWE|Great Exploration...|
|B00HTMCF0U|Harley Quinn in B...|
|B004S8F7QM|Cards Against Hum...|
|B003F25VZO|   Itazura Coin Bank|
|B00000DMBQ|             Cranium|
|B0060Y271K|     Vortex Lumitron|
|B002CS0M7O|Nintendo DS/DSI H...|
|B00DKD324K|Disney Jr. Sofia ...|
|B0032AMW5Q| Queen of Hearts Wig|
|B0015YV6H6|Touch and Feel Fl...|
|B00PZYQODI|Melissa & Doug Me...|
+----------+--------------------+
only showing top 20 rows



In [10]:
product_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [11]:
# Create customer dataframe for the  database
customers_df = toy_df.select(["customer_id"])
customers_df = customers_df.groupBy("customer_id").count()
customers_df.orderBy("customer_id").select(["customer_id", "count"])
customers_df = customers_df.withColumnRenamed("count", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   40305706|             1|
|    3887504|             4|
|   41739251|             1|
|   28952249|            51|
|   34724923|             1|
|   14225559|             3|
|   26285304|             2|
|     167919|             1|
|   26815475|             9|
|   14908291|             6|
|   27672367|             2|
|   24219298|             4|
|   48356484|            27|
|   18461577|             4|
|   46086404|             1|
|   49996826|             7|
|   20159319|             7|
|   24353265|             2|
|   19833133|             4|
|   51567618|             2|
+-----------+--------------+
only showing top 20 rows



In [12]:
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [13]:
# Create vine dataframe for the  database
vine_df = toy_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R10010KWXLSPF3|          5|            0|          0|   N|
|R100BG8WSTAYPG|          5|            1|          1|   N|
|R100F00UQBXLAL|          5|            0|          0|   N|
|R1010FUSHO8ASL|          5|            0|          1|   N|
|R101LBRJ366XL1|          5|            0|          0|   N|
|R101LKRSNW6OIK|          5|            0|          0|   N|
|R101TEA798DC3M|          4|            1|          1|   N|
|R101UKS45S1Z0O|          5|            0|          0|   N|
|R101Y71FKFPSEO|          5|            0|          0|   N|
|R102ETLR4XJZFM|          5|            0|          0|   N|
|R102IJ4R9UE6QH|          5|            0|          0|   N|
|R1038ZA60A3YYJ|          5|            0|          0|   N|
|R1039LT1Z5D6A1|          4|            0|          0|   N|
| R103HL2RILQNH|          5|           1

In [14]:
vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



Postgres Setup

In [15]:
# IMPORTANT: Replace each of these parameters with your own values for your AWS RDS instance
my_aws_endpoint = '' # replace with your own
my_aws_port_number = '' # Your value is likely the same, but please double check
my_aws_database_name = '' # please replace with your own
my_aws_username = '' # Your value is likely the same, but please double check
my_aws_password = ''# replace with your own


In [16]:
# Define the connection string
jdbc_url=f'jdbc:postgresql://{my_aws_endpoint}:{my_aws_port_number}/{my_aws_database_name}'

# Set up the configuration parameters
config = {"user": f'{my_aws_username}', 
          "password": f'{my_aws_password}', 
          "driver":"org.postgresql.Driver"}

# Choose to overwrite the existing data. Note that 'append' is probably a smarter choice
# in those situations where the PostgreSQL schema automatically generates the primary key. 
#  But 'overwrite' works best for this little demo. 
mode = 'overwrite' 


In [17]:
# Write the dataframe to the appropriate table in your PostgreSQL RDS

review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [18]:
# Write the dataframe to the appropriate table in your PostgreSQL RDS

product_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [19]:
# Write the dataframe to the appropriate table in your PostgreSQL RDS

customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [21]:
# Write the dataframe to the appropriate table in your PostgreSQL RDS

vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)